# Векторные хранилища и ретриверы

В этом разделе вы узнаете как использовать GigaChain для работы с векторными хранилищам и ретриверами.
Ретриверы и векторные хранилища важны для приложений, которые извлекают данные для рассуждений в рамках инференса модели, например, в случае генерации с дополнением извлечения данных или RAG (подробнее — в разделе [Создание RAG-приложения](/ru/gigachain/tutorials/rag)).

## Основные понятия

В этом разделе основное внимание уделяется извлечению текстовых данных.
Здесь рассмотрены понятия:

- документы;
- векторные хранилища;
- ретриверы.

## Подготовка к разработке

### Jupyter-блокноты

Как и большинство других руководств в документации, этот раздел использует [Jupyter-блокнот](https://jupyter.org/).
Блокноты отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями, например недоступностью API или нетипичным выводом.

Подробнее об установке jupyter — в [официальной документации](https://jupyter.org/install).

### Установка

Установить пакеты, которые понадобятся для работы с примерами:

In [ ]:
pip install gigachain-community gigachain-chroma

Подробнее об установке GigaChain — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью GigaChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в блокноте, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->


## Документы

GigaChain реализует класс [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html), который представляет единицу текста и связанные с ним метаданные.
У этого класса есть два атрибута:

- `page_content` — строка, представляющая содержимое;
- `metadata` — словарь, содержащий произвольные метаданные.

Атрибут `metadata` может содержать данные об источнике документа, его связи с другими документами и другую дополнительную информацию.
Отдельный объект `Document` часто представляет фрагмент более крупного документа.

Создайте несколько образцов документов:

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Кошки — независимые животные, которым нужно собственное пространство.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Золотые рыбки — отличные домашние животные для начинающих. За ними достаточно просто ухаживать.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Попугаи — умные птицы, которые способны имитировать человеческую речь.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Кролики — социальные животные, которым нужно много места, чтобы прыгать.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

В примере выше представленно пять документов с долполнительными данными, которые указывают на три различных источника (`source`).

## Векторные хранилища

Векторный поиск — это распространенный способ поиска по сохраненным неструктурированным данным, например, неструктурированному тексту.
В основе поиска лежит работа с сохраненными числовыми векторами, которые ассоциированы с текстом.
При запросе к хранилищу, вы можете [представить текст запроса в виде вектора](/docs/concepts#embedding-models) той же размерности и использовать метрики векторной схожести для поиска подходящих данных в хранилище.

Объекты [`VectorStore`](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) предоставляют методы для добавления текста и объектов `Document` в хранилище, а так же методы для поиска документов в хранилище с использованием различных метрик схожести.
Как правило, хранилища инициализируются с использованием моделей, которые поддерживают [векторное представление текста](/ru/gigachain/how-to/embed_text).
Такие модели определяют, как текстовые данные переводятся в числовые векторы.

GigaChain предоставляет набор [интеграций](/docs/integrations/vectorstores) с различными векторными хранилищами.
Некоторые из них могут требовать соответствующие учетные данные для использования.
Другие, такие как [Postgres](/docs/integrations/vectorstores/pgvector), работают в отдельной инфраструктуре, которую можно запускать локально или через сторонние сервисы.
Какие-то могут работать в памяти и используются для легких нагрузок.

В этом разделе показана работа объектов `VectorStore` с использованием [Chroma](/docs/integrations/vectorstores/chroma) — векторным хранилищем, которое поддерживает работу в памяти.

Как правило, для создания векторного хранилища нужно предоставить модель, которая будет отвечать за [векторное представление текста](/ru/gigachain/how-to/embed_text).

Примере для создания векторного представления текста используется модель [GigaChat Embeddings](/ru/gigachat/models#model-dlya-vektornogo-predstavleniya-teksta).

In [3]:
from langchain_chroma import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>",
        scope="GIGACHAT_API_PERS",
        verify_ssl_certs=False,
    ),
)

Вызов метода `.from_documents` добавляет документы в векторное хранилище.
Объект [`VectorStore`](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) предоставляет методы для добавления документов, которые также можно вызвать после создания объекта.
Большинство интеграций позволяет подключаться к существующему векторному хранилищу, например, предоставив клиент, название индекса или другую информацию.
Подробнее — в документации по [интеграции с выбранным векторным хранилищем](/docs/integrations/vectorstores).

После создания объекта `VectorStore` с необходимыми документами, вы можете выполнять поиск поним.
Объект [VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) включает методы для выполнения запросов:
- синхронно и асинхронно;
- по текстовому запросу и по его вектору;
- с возвращением и без возвращения оценок схожести;
- по схожести и [максимальной маржинальной релевантности](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search), которая позволяет сбалансировать схожесть с разнообразием в извлеченных результатах.

В общем случае, результат работы методов будет содержать объектов [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document).

### Примеры

Возврат документов на основе схожести с текстовым запросом:

In [4]:
vectorstore.similarity_search("кошка")

[Document(page_content='Кошки — независимые животные, которым нужно собственное пространство.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Кролики — социальные животные, которым нужно много места, чтобы прыгать.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Попугаи — умные птицы, которые способны имитировать человеческую речь.', metadata={'source': 'bird-pets-doc'})]

Асинхронный запрос:

In [5]:
await vectorstore.asimilarity_search("кошка")

[Document(page_content='Кошки — независимые животные, которым нужно собственное пространство.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Кролики — социальные животные, которым нужно много места, чтобы прыгать.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Попугаи — умные птицы, которые способны имитировать человеческую речь.', metadata={'source': 'bird-pets-doc'})]

Оценка схожести запроса и содержимого хранилища:

In [6]:
# Оценка зависит от выбранного векторного хранилища.
# Chroma возвращает метрику расстояния, которая должна варьироваться обратно пропорционально схожести.

vectorstore.similarity_search_with_score("кошка")

[(Document(page_content='Кошки — независимые животные, которым нужно собственное пространство.', metadata={'source': 'mammal-pets-doc'}),
  218.2356719970703),
 (Document(page_content='Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.', metadata={'source': 'mammal-pets-doc'}),
  319.75384521484375),
 (Document(page_content='Кролики — социальные животные, которым нужно много места, чтобы прыгать.', metadata={'source': 'mammal-pets-doc'}),
  349.84930419921875),
 (Document(page_content='Попугаи — умные птицы, которые способны имитировать человеческую речь.', metadata={'source': 'bird-pets-doc'}),
  352.6993103027344)]

Возврат документов на основе схожести с запросом, представленным в виде вектора:

In [9]:
embedding = GigaChatEmbeddings(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False,
).embed_query("кошка")

vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='Кошки — независимые животные, которым нужно собственное пространство.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Кролики — социальные животные, которым нужно много места, чтобы прыгать.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Попугаи — умные птицы, которые способны имитировать человеческую речь.', metadata={'source': 'bird-pets-doc'})]

Больше информации о векторных хранилищах:

- [Справка API](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html)
- [Работа с векторными хранилищами](/ru/gigachain/how-to/vectorstores)
- [Документация для интеграции с выбранным хранилищем](/docs/integrations/vectorstores)

## Ретриверы

Объекты `VectorStore` не являются [Runnable-объектами](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.runnables) и поэтому их нельзя использовать в [LCEL-цепочках](/docs/concepts/#langchain-expression-language-lcel) напрямую.

В то же время ретриверы GigaChain ([`Retrievers`](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.retrievers)) — являются экземплярами runnable, поэтому они реализуют стандартный набор методов (например, синхронные и асинхронные операции `invoke` и `batch`) и предназначены для включения в цепочки LCEL.

Вы можете самостоятельно создать ретривер, не прибегая к классу `Retriever`.
Для этого нужно выбрать метод, который будет использоваться для извлечения документов, и создать runnable.
Пример ниже показывает как создать ретривер, который использует метод `similarity_search`, на основе Runnable:

In [10]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(
    k=1
)  # выбор наиболе подходящего результата

retriever.batch(["кошка", "акула"])

[[Document(page_content='Кошки — независимые животные, которым нужно собственное пространство.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.', metadata={'source': 'mammal-pets-doc'})]]

Векторные хранилища предоставляют метод `as_retriever`, который создаст экземпляр класса Retriever, а именно [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html).
Эти ретриверы включают заданные атрибуты `search_type` и `search_kwargs`, которые определяют, какие методы базового векторного хранилища вызывать и как задавать их параметры.
Так, вы можете повторить функциональность из примера выше с помощью следующего кода:

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["кошка", "акула"])

[[Document(page_content='Кошки — независимые животные, которым нужно собственное пространство.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Собаки — отличные компаньоны, которые известны своей преданностью и дружелюбием.', metadata={'source': 'mammal-pets-doc'})]]

`VectorStoreRetriever` поддерживает типы поиска `"similarity"` (по умолчанию), `"mmr"` (maximum marginal relevance, описано выше) и `"similarity_score_threshold"`.
Последний тип можно использовать для отсечки документов, выводимых ретривером, на основе оценки схожести.

Ретриверы можно легко включить в более сложные RAG-приложения, которые объединяют заданный вопрос с извлеченным контекстом в промпт для модели.

In [12]:
# | output: false
# | echo: false

from langchain_community.chat_models.gigachat import GigaChat

model = GigaChat(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Отвечай на вопросы только с помощью полученного контекста.

{question}

Контекст:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

In [14]:
response = rag_chain.invoke("Расскажи о кошках")

print(response.content)

Кошки - это независимые животные, которые нуждаются в своем собственном пространстве.


## Смотрите также

Стратегии извлечения данных могут быть разнообразными и сложными. Например:

- [Выведение жестких правила и фильтры](/ru/gigachain/how-to/self_query) из запроса. Например, «использовать документы, опубликованные после 2020 года»;
- [Возвращение документов, каким-то образом связанных с извлеченным контекстом](/ru/gigachain/how-to/parent_document_retriever). Например, через таксономию документов;
- [Создание нескольких векторных представлений](/ru/gigachain/how-to/multi_vector) для каждой единицы контекста;
- [Объединение результаты](/ru/gigachain/how-to/ensemble_retriever) от нескольких ретриверов;
- [Управление весами документов](/ru/gigachain/how-to/time_weighted_vectorstore). Например, придание большего веса недавним документам.

Раздел [Работа с ретриверами](/docs/how_to#retrievers) в руководствах по использованию охватывает эти и другие стратегии извлечения данных.

Вы также можете расширить класс [BaseRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html) для реализации [собственных ретриверов](/ru/gigachain/how-to/custom_retriever).